In [5]:
# jupyter nbconvert --to script {파일명}.ipynb
#jupyter nbconvert --to script casca_train.ipynb
# ipynb -> py
import torch
torch.cuda.empty_cache()

In [6]:
!nvidia-smi

Mon Jan  8 02:57:28 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   30C    P0    64W / 300W |  25876MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device
import wandb
import torch

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

wandb.init(project='cascade', name='cascade641x_weighted')
# config file 들고오기

cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_1x_coco.py')
#cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_20e_coco.py')

root='../../../dataset/'

#weighted_cross_entropy_loss = torch.tensor([2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]).cuda()
# 1General trash / 2Paper / 3Paper pack / 4Metal / 5Glass / 6Plastic /7Styrofoam / 8Plastic bag / 9Battery / 10Clothing
#cfg.model.rpn_head.loss_cls.loss_weight = weighted_cross_entropy_loss            

cfg.model.rpn_head.loss_cls.type = 'FocalLoss'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보

cfg.data.samples_per_gpu = 8

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head[0].num_classes = 10
cfg.model.roi_head.bbox_head[1].num_classes = 10
cfg.model.roi_head.bbox_head[2].num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project':'cascade'},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100)
]


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: edwin-minlee (level2-cv-10-detection). Use `wandb login --relogin` to force relogin


In [3]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


In [4]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

/data/ephemeral/home/level2-objectdetection-cv-10/develop/yumin/mmdetection/mmdet/models/dense_heads/anchor_head.py:93: UserWarning: DeprecationWarning: Determining whether to samplingby loss type is deprecated, please delete sampler inyour config when using `FocalLoss`, `GHMC`, `QualityFocalLoss` or other FocalLoss variant.
  warnings.warn(


AssertionError: CascadeRCNN: CascadeRoIHead: Shared2FCBBoxHead: FocalLoss: Only sigmoid focal loss supported now.

In [ ]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-01-08 02:39:18,601 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-01-08 02:39:18,604 - mmdet - INFO - Start running, host: root@instance-5158, work_dir: /data/ephemeral/home/level2-objectdetection-cv-10/develop/yumin/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash
2024-01-08 02:39:18,606 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                 

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


2024-01-08 02:39:52,902 - mmdet - WARNING - No meta information found in the runner. 
2024-01-08 02:39:52,903 - mmdet - WARNING - To log evaluation or checkpoint metadata in MMDetWandbHook, `EvalHook` or `DistEvalHook` in mmdet is required, please check whether the validation is enabled.
2024-01-08 02:41:19,007 - mmdet - INFO - Epoch [1][50/611]	lr: 9.890e-05, eta: 4:21:35, time: 1.722, data_time: 0.072, memory: 22127, loss_rpn_cls: 0.7198, loss_rpn_bbox: 0.0364, s0.loss_cls: 1.1149, s0.acc: 74.3799, s0.loss_bbox: 0.0627, s1.loss_cls: 0.6162, s1.acc: 74.4551, s1.loss_bbox: 0.0207, s2.loss_cls: 0.3447, s2.acc: 74.3799, s2.loss_bbox: 0.0033, loss: 2.9188, grad_norm: 17.8922
2024-01-08 02:42:41,651 - mmdet - INFO - Epoch [1][100/611]	lr: 1.988e-04, eta: 4:14:56, time: 1.653, data_time: 0.019, memory: 22127, loss_rpn_cls: 0.2675, loss_rpn_bbox: 0.0376, s0.loss_cls: 0.4012, s0.acc: 93.5342, s0.loss_bbox: 0.1555, s1.loss_cls: 0.1098, s1.acc: 96.9214, s1.loss_bbox: 0.0507, s2.loss_cls: 0.0350

In [ ]:
!nvidia-smi